In [1]:
#ランタイムのタイプをGPUに変更
!pip install langchain
!pip install -U langchain-community
!pip install sentence-transformers
!pip install -U --quiet langchain-google-genai pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/2

In [2]:
import os
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [12]:
import pickle
path_w1 = '/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_texts.pkl'
with open(path_w1, mode='rb') as f:
    texts = pickle.load(f)
path_w2 = '/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_metadatas.pkl'
with open(path_w2, mode='rb') as f:
    metadatas = pickle.load(f)

In [5]:
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

<ipython-input-5-7d781a8d0969>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
 

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [6]:
!pip uninstall faiss-cpu

In [7]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.0 MB/s eta 0:00:00


In [14]:
vectorstore = FAISS.from_texts(texts,embedding, metadatas)
#実行は3分ほど

In [15]:
vectorstore.save_local('/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_vectorstore')
#vectorestoreを保存

In [16]:
vectorstore = FAISS.load_local('/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_vectorstore', embedding,allow_dangerous_deserialization=True)
#vectorestoreを読み込み

In [19]:
retriever = vectorstore.as_retriever()
#retrieverの指定　デフォルトは類似度検索
#引数をsearch_kwargs={'filter': {'授業形態':'講義'}}とすると絞り込める
template = """
ユーザー:Questionに当てはまる科目を複数個挙げて、それぞれの科目の概要を説明してください。

{context}

{question}

システム:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
    template_format="f-string"
)
#プロンプトの作成

os.environ['GOOGLE_API_KEY'] = '************'
#https://ai.google.dev/からAPIのキーを取得
llm = ChatGoogleGenerativeAI(model='gemini-pro')
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
)

q = "機械学習について学べる科目はなんですか？"
ans = qa.invoke(q)

print(ans['result'])

docs = ans['source_documents']
for d in docs:
    print()
    print(d.metadata['科目名'])
    print(d.page_content)


> Finished chain.
* パターン認識と機械学習

パターン認識と機械学習
科目名はパターン認識と機械学習。授業の概要・目的は 実世界のデータから有用な情報を抽出し、カテゴリ分類や予測を行うためのモデル・システムについて講義する。特に、パターン認識を指向した種々のモデル及び機械学習の方法を講義する。到達目標は 機械学習の基本的なアプローチ及び主要な方法について修得する。実世界のパターンを分類・認識するシステムを設計できるようになる。授業計画と内容は 1. パターン認識入門2. 識別関数と機械容量3. 正規分布に基づく識別関数の構成4. クラスタリングと混合正規分布5. DPマッチングとHMM（時系列パターンの認識）6. ベイズ決定論、及び最尤推定とベイズ推定7. ナイーブベイズ識別器とロジスティック回帰モデル8. パーセプトロンとサポートベクトルマシン9. ニューラルネットワーク10. 統計的特徴抽出11. 深層学習12. RNNとTransformer13. パターン認識システムと教師つきでない学習14. パターン認識の最近の話題15. 試験・フィードバック。履修要件は 特になし 。成績評価の方法・観点は 期末試験授業中に小テストを実施する場合あり。授業外学修（予習・復習）等は 講義スライドに演習課題あり  

数理工学概論
科目名は数理工学概論。授業の概要・目的は データサイエンス，アルゴリズム・離散数学，オペレーションズリサーチなどを題材として，モデリングや解析，設計における数理工学の基本的な考え方を解説する．。到達目標は 数理工学の基本的な考え方を理解する．。授業計画と内容は 数理モデリングと非線形科学（寺前），4回，我々を取り巻く物理，生命，社会現象などの多くが相互作用する多数の要素が生み出す集団現象として記述される．その数理構造を解明するための非線形科学の基礎を紹介する．特に，同期現象，相転移，神経情報処理，脳の学習などを取り上げ，数理モデルの果たす役割をみる． データサイエンスの数理（本多），4回，大規模言語モデルや画像生成モデルからウェブサービスの推薦システムまで，AI技術は近年さまざまな場面で応用されている．その基盤となる機械学習の基礎技術について紹介し，確率論や統計学、最適化といった分野との関連について説明する．  逐次的意思決定